# Albeno effekt
---


#### Kode 1
Her ser vi igjen på utviklingen i gjennomsnittlig albedo over tid for faste punkter med høy albedo og lav feilmargin, basert på data fra 2004 til 2024. Det er laget en prediksjonsmodell ved hjelp av lineær regresjon, som estimerer albedo de neste fem årene. Tre ulike visualiseringer linjediagram, søylediagram og scatterplot er implementert for å formidle både historiske trender og fremtidige estimater. Brukeren kan interaktivt velge for å utforske dataene visuelt.


In [8]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from IPython.display import display

# Leser inn referansedata for 2004
referanse_2004 = pd.read_csv("../data/csv_albedo_effekt/Albedo effekt 2004.csv")
feilmargin = pd.read_csv("../data/csv_albedo_effekt/data_m-lavfeilmargin.csv")

# Filtrerer ut punkter med høy albedo og lav feilmargin (AL-BB-DH > 0.3)
felles_2004 = pd.merge(referanse_2004, feilmargin, on=["lat", "lon"], how="inner")
referanse_punkter = felles_2004[felles_2004["AL-BB-DH"] > 0.3][["lat", "lon"]]

# Leser inn alle årlige datafiler og sorterer dem etter årstall
datafiler = sorted(glob.glob("../data/csv_albedo_effekt/Albedo*.csv"))

# Lister for lagring av statistikk
årstall_liste = []
gjennomsnitt_liste = []

# Går gjennom hvert år og beregner gjennomsnitt for faste punkter
for fil in datafiler:
    df = pd.read_csv(fil)
    felles = pd.merge(referanse_punkter, df, on=["lat", "lon"], how="inner")

    if not felles.empty and "AL-BB-DH" in felles.columns:
        gjennomsnitt = felles["AL-BB-DH"].mean()
    else:
        gjennomsnitt = np.nan

    år = int(''.join(filter(str.isdigit, os.path.basename(fil))))
    årstall_liste.append(år)
    gjennomsnitt_liste.append(gjennomsnitt)

# Oppretter en DataFrame med årlig gjennomsnitt
stats = pd.DataFrame({"År": årstall_liste, "Gjennomsnittlig albedo": gjennomsnitt_liste}).dropna()

# Bygger lineær regresjonsmodell for prediksjon
X = stats[["År"]]
y = stats["Gjennomsnittlig albedo"]

model = LinearRegression()
model.fit(X, y)

# Lager fremtidig prediksjon for 5 år
framtid = pd.DataFrame({"År": np.arange(stats["År"].max() + 1, stats["År"].max() + 6)})
framtid["Predikert"] = model.predict(framtid)

# Visualiserer som linjediagram med trendlinje og prediksjon
def vis_linje():
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=stats, x="År", y="Gjennomsnittlig albedo", marker="o", label="Historisk")
    sns.lineplot(data=framtid, x="År", y="Predikert", marker="o", linestyle="--", color="orange", label="Predikert")
    sns.regplot(data=stats, x="År", y="Gjennomsnittlig albedo", scatter=False, color="gray", ci=None, label="Trend")
    plt.title("Linjediagram: Gjennomsnittlig albedo med trendlinje og prediksjon")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Visualiserer som søylediagram med fremtidsverdier og trend
def vis_soyle():
    plt.figure(figsize=(10, 5))
    plt.bar(stats["År"], stats["Gjennomsnittlig albedo"], color="skyblue", label="Historisk")
    plt.bar(framtid["År"], framtid["Predikert"], color="orange", alpha=0.6, label="Predikert")
    sns.regplot(data=stats, x="År", y="Gjennomsnittlig albedo", scatter=False, color="black", ci=None, label="Trend", line_kws={"linestyle": "--"})
    plt.title("Søylediagram: Gjennomsnittlig albedo med trend og prediksjon")
    plt.xlabel("År")
    plt.ylabel("Albedo")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Visualiserer som scatterplot med fremtid og trend
def vis_spredning():
    plt.figure(figsize=(10, 5))
    sns.scatterplot(data=stats, x="År", y="Gjennomsnittlig albedo", s=80, label="Historisk")
    sns.scatterplot(data=framtid, x="År", y="Predikert", s=80, color="orange", label="Predikert")
    sns.regplot(data=stats, x="År", y="Gjennomsnittlig albedo", scatter=False, color="gray", ci=None, label="Trend")
    plt.title("Scatterplot: Historiske punkter og prediksjon med trendlinje")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Interaktiv meny med tre valg for visualisering
def vis_plot(valg):
    if valg == "Linjediagram":
        vis_linje()
    elif valg == "Søylediagram":
        vis_soyle()
    elif valg == "Scatterplot":
        vis_spredning()

meny = widgets.Dropdown(
    options=["Linjediagram", "Søylediagram", "Scatterplot"],
    value="Linjediagram",
    description="Velg visning:"
)

widgets.interact(vis_plot, valg=meny)


interactive(children=(Dropdown(description='Velg visning:', options=('Linjediagram', 'Søylediagram', 'Scatterp…

<function __main__.vis_plot(valg)>